In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    BatchNormalization,
    Add,
    MaxPool2D,
)
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential, Model

seed_value = 42

np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls ./kaggle/working

## Exploration, Analyse, Data Preprocessing

In [ ]:
directory = "./kaggle/working/ships32"
batch_size = 128
img_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    image_size=(img_size, img_size),
    batch_size=batch_size,
    shuffle=True,
    validation_split=0.2,
    seed=42,
    subset="training",
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    image_size=(img_size, img_size),
    batch_size=batch_size,
    shuffle=True,
    validation_split=0.2,
    seed=42,
    subset="validation",
)


In [ ]:
class_counts = {}
for image, label in train_ds.unbatch():
    label_val = label.numpy()  # Get the actual label value
    if label_val in class_counts:
        class_counts[label_val] += 1
    else:
        class_counts[label_val] = 1

# Plot class distribution
plt.figure(figsize=(12, 6))
classes = list(class_counts.keys())
counts = list(class_counts.values())
plt.bar(classes, counts)
plt.xlabel("Boat Class")
plt.ylabel("Number of Images")
plt.title("Class Distribution in Training Dataset")
plt.show()

In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomTranslation(0.1, 0.1),
        tf.keras.layers.RandomContrast(0.2),
        tf.keras.layers.RandomSaturation(0.2),
    ]
)

normalization_layer = layers.Rescaling(1.0 / 255)

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Model

In [ ]:
def BoatNet_30(input_shape=(32, 32, 3), num_classes=13):
    inputs = Input(shape=input_shape)

    # Block 1
    x = Conv2D(
        64, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(
        64, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Block 2
    x = Conv2D(
        128, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(x)
    x = BatchNormalization()(x)
    x = Conv2D(
        128, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    # === Residual block (3 layers) ===
    # Shortcut before main path
    shortcut = Conv2D(128, (1, 1), padding="same")(x)  # 1
    # Main path
    res = Conv2D(128, (3, 3), padding="same", activation="relu")(x)  # 2
    res = BatchNormalization()(res)  # 3
    # Add skip connection
    x = Add()([shortcut, res])

    # Block 3
    x = Conv2D(
        256, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(x)
    x = BatchNormalization()(x)
    x = Conv2D(
        256, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.4)(x)

    # Block 4
    x = Conv2D(
        512, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    # Classification
    x = GlobalAveragePooling2D()(x)
    x = Dense(768, activation="relu", kernel_regularizer=l2(2e-4))(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes)(x)  # from_logits=True for sparse loss

    return Model(inputs=inputs, outputs=outputs)


In [ ]:
model = BoatNet_30()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

print("Nombre de couches : ", len(model.layers))

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.3,
    patience=3,
    min_lr=1e-6,
    verbose=1,
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

epochs = 50

history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[reduce_lr, early_stopping])

## Résultat à soumettre

In [ ]:
X_test = np.load("./kaggle/input/navires-2025/ships_competition.npz", allow_pickle=True)[
    "X"
]
X_test = X_test.astype("float32") / 255


In [ ]:
res = model.predict(X_test).argmax(axis=1)
df = pd.DataFrame({"Category": res})
df.to_csv("reco_nav.csv", index_label="Id")


In [ ]:
!head reco_nav.csv


In [ ]:
import os

os.chdir(r"./kaggle/working")
from IPython.display import FileLink

FileLink(r"reco_nav.csv")
